In [2]:
import sys
sys.path.append("../") 

%load_ext autoreload
%autoreload 2

import os
import os.path as osp

from torch.utils.data import random_split

from nscl.datasets.clevr_dataset import build_clevr_dataset, build_clevr_dataloader, CLEVRCurriculumSampler
from nscl.datasets.clevr_definition import CLEVRDefinition, QuestionTypes
from nscl.models.nscl_module import NSCLModule


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
train_img_root = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/images/train'
train_scene_json = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/scenes/train/scenes.json'
train_question_json = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/questions/CLEVR_train_questions.json'

In [4]:
batch_size = 100
num_workers = 4
dataset = build_clevr_dataset(train_img_root, train_scene_json, train_question_json)
train_dataset, val_dataset = random_split(dataset, [len(dataset) - len(dataset)//3, len(dataset)//3])

loading scenes from: /home/vs2n19/mydocuments/NSCL-reproduce/data/CLEVR_v1.0/scenes/train/scenes.json
loading questions from: /home/vs2n19/mydocuments/NSCL-reproduce/data/CLEVR_v1.0/questions/CLEVR_train_questions.json


In [6]:
train_curriculum_sampler = CLEVRCurriculumSampler(train_dataset, max_scene_size=5, max_program_size=5, max_data_size=1000)
val_curriculum_sampler = CLEVRCurriculumSampler(val_dataset, max_scene_size=5, max_program_size=5, max_data_size=100)

train_loader = build_clevr_dataloader(train_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False,
                                     drop_last=False, sampler=train_curriculum_sampler)
val_loader = build_clevr_dataloader(val_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False,
                                     drop_last=False, sampler=val_curriculum_sampler)

Preparing curriculum sampler....
Preparing curriculum sampler....


In [18]:
import torch
from torch import nn
from torch import optim
from tqdm.autonotebook import tqdm

device = "cuda:0" if torch.cuda.is_available() else "cpu"
epoch = 10
model = NSCLModule(CLEVRDefinition.attribute_concept_map).to(device)
mse_loss = nn.MSELoss()
bce_loss = nn.BCELoss()
ce_loss = nn.CrossEntropyLoss()

opt = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=0.01)

for e in range(epoch):
    model.train()
    with tqdm(total=len(train_loader), desc='train') as t:
        epoch_loss = 0
        for idx, (images, questions, scenes) in enumerate(train_loader):
            opt.zero_grad()
            results = model(images.to(device), questions, scenes)
            total_loss = torch.tensor(0.0, requires_grad=True, dtype=torch.float, device=device)
            for i, q in enumerate(questions):
                if q.question_type == QuestionTypes.COUNT:
                    loss = mse_loss(results[i], q.answer_tensor.to(device))
                elif q.question_type == QuestionTypes.BOOLEAN:
                    loss = bce_loss(results[i], q.answer_tensor.to(device))
                else:
                    loss = bce_loss(results[i], q.answer_tensor.to(device))

                total_loss = total_loss + loss

            total_loss.backward()
            opt.step()
            epoch_loss = (epoch_loss * idx + total_loss.item()) / (idx + 1)
            t.set_postfix(loss='{:05.3f}'.format(epoch_loss))
            t.update()

    model.eval()
    with tqdm(total=len(val_loader), desc='val') as t:
        with torch.no_grad():
            epoch_loss = 0
            for idx, (images, questions, scenes) in enumerate(val_loader):
                results = model(images.to(device), questions, scenes)
                total_loss = torch.tensor(0.0, requires_grad=True, dtype=torch.float, device=device)
                for i, q in enumerate(questions):
                    if q.question_type == QuestionTypes.COUNT:
                        loss = mse_loss(results[i], q.answer_tensor.to(device))
                    elif q.question_type == QuestionTypes.BOOLEAN:
                        loss = bce_loss(results[i], q.answer_tensor.to(device))
                    else:
                        loss = bce_loss(results[i], q.answer_tensor.to(device))

                    total_loss = total_loss + loss

                epoch_loss = (epoch_loss * idx + total_loss.item()) / (idx + 1)
                t.set_postfix(loss='{:05.3f}'.format(epoch_loss))
                t.update()

### Testing the Model

In [19]:
batch_size = 100
num_workers = 4
test_img_root = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/images/val'
test_scene_json = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/scenes/val/scenes.json'
test_question_json = osp.abspath(osp.dirname(os.getcwd())) + '/data/CLEVR_v1.0/questions/CLEVR_val_questions.json'

test_dataset = build_clevr_dataset(test_img_root, test_scene_json, test_question_json)
test_curriculum_sampler = CLEVRCurriculumSampler(test_dataset, max_scene_size=5, max_program_size=5, max_data_size=100)
test_loader = build_clevr_dataloader(test_dataset, batch_size=batch_size, num_workers=num_workers, shuffle=False,
                                     drop_last=False, sampler=test_curriculum_sampler)

loading scenes from: /home/vs2n19/mydocuments/NSCL-reproduce/data/CLEVR_v1.0/scenes/val/scenes.json
loading questions from: /home/vs2n19/mydocuments/NSCL-reproduce/data/CLEVR_v1.0/questions/CLEVR_val_questions.json
Preparing curriculum sampler....


In [21]:
model.eval()
correct = 0
count = 0
with tqdm(total=len(val_loader), desc='test') as t:
    with torch.no_grad():
        epoch_loss = 0
        for idx, (images, questions, scenes) in enumerate(test_loader):
            results = model(images.to(device), questions, scenes)
            total_loss = torch.tensor(0.0, dtype=torch.float)
            for i, q in enumerate(questions):
                if q.question_type == QuestionTypes.COUNT:
                    loss = mse_loss(results[i], q.answer_tensor.to(device))
                    print(q.raw_question)
                    print('True Answer', q.answer)
                    print('Predicted Answer', int(round(results[i].item())))
                    if int(round(results[i].item())) == q.answer:
                        correct += 1
                elif q.question_type == QuestionTypes.BOOLEAN:
                    loss = bce_loss(results[i], q.answer_tensor.to(device))
                    print(q.raw_question)
                    print('True Answer', q.answer)
                    print('Predicted Answer', 'no' if int(round(results[i].item())) == 0 else 'yes')
                    if int(round(results[i].item())) == int(q.answer_tensor.item()):
                        correct += 1
                else:
                    loss = bce_loss(results[i], q.answer_tensor.to(device))
                    print(q.raw_question)
                    print('True Answer', q.answer)
                    print('True Answer distribution', torch.argmax(q.answer_tensor).item())
                    print('Predicted Answer', torch.argmax(results[i]).item())
                    if torch.argmax(results[i]).item() == torch.argmax(q.answer_tensor).item():
                        correct += 1
                count += 1
                total_loss = total_loss + loss

            epoch_loss = (epoch_loss * idx + total_loss.item()) / (idx + 1)
            t.set_postfix(loss='{:05.3f}'.format(epoch_loss))
            t.update()

print('Test Accuracy', correct/count)

What is the material of the big purple object?
True Answer metal
True Answer distribution 1
Predicted Answer 1
How many small matte things are there?
True Answer 1
Predicted Answer 3
What number of tiny objects are there?
True Answer 3
Predicted Answer 4
What is the color of the rubber cube?
True Answer red
True Answer distribution 1
Predicted Answer 0
What shape is the brown thing?
True Answer cylinder
True Answer distribution 2
Predicted Answer 2
What is the material of the small purple thing?
True Answer rubber
True Answer distribution 0
Predicted Answer 1
The big block has what color?
True Answer gray
True Answer distribution 0
Predicted Answer 0
What size is the sphere?
True Answer small
True Answer distribution 0
Predicted Answer 0
The large brown thing is what shape?
True Answer cube
True Answer distribution 0
Predicted Answer 2
The sphere is what color?
True Answer red
True Answer distribution 1
Predicted Answer 0
What size is the blue object?
True Answer small
True Answer dist